# installing libraries

In [0]:
pip install gwpy Pillow pandas scikit-learn seaborn sklearn

     |████████████████████████████████| 1.4MB 2.7MB/s 
     |████████████████████████████████| 51kB 9.4MB/s 
     |████████████████████████████████| 61kB 10.8MB/s 
     |████████████████████████████████| 2.3MB 50.1MB/s 
  Created wheel for ligo-segments: filename=ligo_segments-1.2.0-cp36-cp36m-linux_x86_64.whl size=83620 sha256=e7b7e818127c3114f5e92985a38ea3adce9a9cd81847071c33749f8567fc1c0c
  Stored in directory: /root/.cache/pip/wheels/5d/1e/4a/ab4122baed7d67f6abce65b2b12049d3bc7fe5dad24edf89df
Successfully built ligo-segments


In [0]:
import seaborn as sns
%matplotlib inline

import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import matplotlib as mlp

import numpy as np

from gwpy.timeseries import TimeSeries
from gwpy.signal import filter_design
from gwpy.segments import DataQualityFlag
from gwpy.plot import Plot
from gwpy.segments import Segment

import torch
import torch.nn as nn
from torch.autograd import Variable
import pandas as pd
import torchvision
import torchvision.transforms as transforms

import PIL
from numpy import asarray
from PIL import Image
from matplotlib import pyplot
import matplotlib.image as mpimg

# directory 

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'fastai-v3/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
ls 'gdrive/My Drive/GW data/Sdata'

10.png  1.png   29.png  38.png  47.png  56.png  65.png  74.png  83.png  92.png
11.png  20.png  2.png   39.png  48.png  57.png  66.png  75.png  84.png  93.png
12.png  21.png  30.png  3.png   49.png  58.png  67.png  76.png  85.png  94.png
13.png  22.png  31.png  40.png  4.png   59.png  68.png  77.png  86.png  95.png
14.png  23.png  32.png  41.png  50.png  5.png   69.png  78.png  87.png  96.png
15.png  24.png  33.png  42.png  51.png  60.png  6.png   79.png  88.png  97.png
16.png  25.png  34.png  43.png  52.png  61.png  70.png  7.png   89.png  98.png
17.png  26.png  35.png  44.png  53.png  62.png  71.png  80.png  8.png   99.png
18.png  27.png  36.png  45.png  54.png  63.png  72.png  81.png  90.png  9.png
19.png  28.png  37.png  46.png  55.png  64.png  73.png  82.png  91.png


In [0]:
from PIL import Image
loc='/content/gdrive/My Drive/GW data/Sdata/1.png'
image = Image.open(loc)
print(image.size)

(9000, 3000)


# labelling image

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving labels.csv to labels.csv
User uploaded file "labels.csv" with length 3495 bytes


In [0]:
t=pd.read_csv('labels.csv')
t.describe

<bound method NDFrame.describe of    event    e    cm     s    t1    m1    t2    m2    tr    mr    sr
0    yes  3.1  28.6 -0.01    BH  35.6    BH  30.6    BH  63.1  0.69
1    yes  3.1  28.6 -0.01    BH  36.6    BH  31.6    BH  63.1  0.69
2    yes  3.1  28.6 -0.01    BH  37.6    BH  32.6    BH  63.1  0.69
3    yes  3.1  28.6 -0.01    BH  38.6    BH  33.6    BH  63.1  0.69
4    yes  3.1  28.6 -0.01    BH  39.6    BH  34.6    BH  63.1  0.69
..   ...  ...   ...   ...   ...   ...   ...   ...   ...   ...   ...
95    no  0.0   0.0  0.00  none   5.0  none   5.0  none   0.0  0.00
96    no  0.0   0.0  0.00  none   6.0  none   6.0  none   0.0  0.00
97    no  0.0   0.0  0.00  none   7.0  none   7.0  none   0.0  0.00
98    no  0.0   0.0  0.00  none   8.0  none   8.0  none   0.0  0.00
99    no  0.0   0.0  0.00  none   9.0  none   9.0  none   0.0  0.00

[100 rows x 11 columns]>

## image processing

In [0]:
train = pd.read_csv('labels.csv')

In [0]:
data=np.zeros((100, 28, 28), dtype=np.float32)
#11600 in z dir
#7030 in y 4 along x

na=[str(a) for a in range(1,100)]
for i in range(100):
  loc='/content/gdrive/My Drive/GW data/Sdata/'+na[i]+'.png'
  image = Image.open(loc)
  cropped = image.crop((1080, 320, 8110, 2640))
  gs_image = cropped.convert(mode='L')
  img = gs_image.resize((28,28))
  pyplot.grid(0)
  pyplot.axis('off')
  #pyplot.imshow(img)
  #pyplot.show()
  data[i]=asarray(img)/255
  #print(data[i])

AttributeError: ignored

In [0]:
X=np.array(data)
print(X.shape)

In [0]:
plt.figure(figsize=(10,5))
plt.scatter(t.m1, t.e)

In [0]:
X=t[['m1','m2']]
y=t['e']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.1,random_state=7)

In [0]:
import torch.nn.functional as F
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)# in channel 1 means 1 color input with 6 channels of filter used to produce output of size 5x5
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)# input for one layer is ouput for another layer
        self.fc1 = nn.Linear(in_features=12 * 4 * 4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)# out_features represents the no. of trining classes
        
    def forward(self, t):
        t=self.conv1(t)#weights fucntion
        t=F.relu(t)# activation function not layers as they donot have any weights attached
        t=F.max_pool2d(t,kernel_size=2,stride=2)
        t=self.conv2(t)
        t=F.relu(t)
        t=F.max_pool2d(t,kernel_size=2,stride=2)
        t=t.reshape(-1,12*4*4)
        t=self.fc1(t)
        t=F.relu(t)
        t=self.fc2(t)
        t=F.relu(t)        
        t=self.out(t)    
        return t

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm

Using TensorFlow backend.


In [0]:
na=[str(a) for a in range(1,102)]
train_image = []
for i in np.arange(1,99):
    img = image.load_img('/content/gdrive/My Drive/GW data/Sdata/'+na[i]+'.png', target_size=(28,28,1), color_mode="grayscale")
    img = image.img_to_array(img)
    img = img/255
    print(i)
    train_image.append(img)
X = np.array(train_image)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98


In [0]:
train = pd.read_csv('labels.csv')

In [0]:
y=train['e'].values
y = to_categorical(y)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=7, test_size=0.1)

ValueError: ignored

In [0]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(28,28,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

In [0]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [0]:
model.fit(X_train, y_train, epochs=500, validation_data=(X_test, y_test))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 18 samples, validate on 2 samples
Epoch 1/500





18/18 [==============================] - 10s 543ms/step - loss: 1.4062 - acc: 0.2778 - val_loss: 0.9287 - val_acc: 1.0000
Epoch 2/500
18/18 [==============================] - 0s 650us/step - loss: 0.9713 - acc: 0.5000 - val_loss: 0.7213 - val_acc: 1.0000
Epoch 3/500
18/18 [==============================] - 0s 728us/step - loss: 0.8865 - acc: 0.3889 - val_loss: 0.6535 - val_acc: 1.0000
Epoch 4/500
18/18 [==============================] - 0s 697us/step - loss: 0.5246 - acc: 0.7222 - val_loss: 0.6325 - val_acc: 0.5000
Epoch 5/500
18/18 [==============================] - 0s 739us/step - loss: 0.9319 - acc: 0.4444 - val_loss: 0.5969 - val_acc: 0.5000
Epoch 6/500
18/18 [==============================] - 0s 698us/step - loss: 0.8219 - acc: 0.4444 - val_loss: 0.5621 - val_acc: 1.0000
Epoch 7/500
18/18 [==============================] - 0s 

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving labels.csv to labels (1).csv
User uploaded file "labels.csv" with length 3495 bytes


In [0]:
test=pd.read_csv("labels.csv")

In [0]:
na=[str(a) for a in range(51,70,1)]
test_image = []
for i in np.arange(5):
    img = image.load_img('/content/gdrive/My Drive/GW data/Sdata/'+na[i]+'.png', target_size=(28,28,1), color_mode="grayscale")
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
test = np.array(test_image)

In [0]:
prediction = model.predict_classes(test)

In [0]:
prediction

array([15, 15,  2,  9,  2])